In [1]:
import pandas as pd
import requests
import numpy as np
import bs4
import stringdist
import re

# UK Enterococcus Paper Metadata

## Add sample and project accessions to UK metadata

Starting from table S2 in https://mbio.asm.org/content/9/6/e01780-18 which contains `run_accessions` for deposited raw sequencing data in ENA, use the ENA API to automatically add `study_accession` and `sample_accession`


In [2]:
# table S2 from https://mbio.asm.org/content/9/6/e01780-18
uk_metadata = pd.read_csv('inline-supplementary-material-3.csv', sep='\t', skiprows=1)
uk_metadata = uk_metadata.rename(columns={'Accession number': 'Run_accession', 
                                          'Isolate ID': 'Isolate_name'})

# break into 500 accession chunks to more efficiently query the API
api_responses = ""

run_accessions = uk_metadata.loc[uk_metadata['Run_accession'].str.startswith('ERR'), 'Run_accession'].values
for run_accession_chunk in [run_accessions[i:i + 500] for i in range(0, len(run_accessions), 500)]:
    run_accession_chunk = ",".join(run_accession_chunk)
    api_response = requests.get(f"https://www.ebi.ac.uk/ena/browser/api/xml/{run_accession_chunk}")    
    api_responses += api_response.text

# parse combined xml records into a soup for easier traversal 
api_response_soup = bs4.BeautifulSoup(api_responses, 'lxml')


def add_study_and_sample_metadata(row, api_response_soup):
    """
    Use the ENA API to get the study and sample accessions
    """
    run_accession = row['Run_accession']
        
    run_soup = api_response_soup.find(accession=run_accession)
    if run_soup:
        for xref in run_soup.find_all('xref_link'):
            if xref.db.text == 'ENA-STUDY':
                row['Study_accession'] = xref.id.text.strip()
            elif xref.db.text == 'ENA-SAMPLE':
                row['Sample_accession'] = xref.id.text.strip()
    else:
        row['Study_accession'] = np.nan
        row['Sample_accession'] = np.nan
    return row

uk_metadata = uk_metadata.apply(lambda x: add_study_and_sample_metadata(x, api_response_soup), axis=1)

Then we want to clean up the column names to make merging easier

In [3]:
# tidy up UK metadata for merging
uk_metadata = uk_metadata.rename(columns={'Origin': 'Origin',
                                          'BAPS group': 'BAPS_group',
                                          'Location': 'Location',
                                          'Ampicillin resistance': 'Ampicillin',
                                          'Vancomycin resistance': 'Vancomycin'})

#uk_metadata.loc[uk_metadata['Removed in deduplication'] == 'removed', 'Status'] = 'Removed for deduplication in original paper (10.1128/mBio.01780-18)'
#uk_metadata = uk_metadata.drop('Removed in deduplication', axis=1)

# all in this paper are E. faecium
uk_metadata['Species'] = 'Enterococcus faecium'
uk_metadata['Country/province'] = 'United Kingdom'

In [4]:
uk_metadata

,Isolate_name,Run_accession,Origin,BAPS_group,ST,Location,Ampicillin,Vancomycin,Removed in deduplication,Study_accession,Sample_accession,Species,Country/province
0,10678_8#11,ERR369948,Human invasive infection,2,117,Hospital 34,R,R,NaN,ERP003621,ERS325719,Enterococcus faecium,United Kingdom
1,10678_8#14,ERR369951,Human invasive infection,1,117,Hospital 1,R,R,NaN,ERP003621,ERS325652,Enterococcus faecium,United Kingdom
2,10678_8#17,ERR369954,Human invasive infection,1,186,Hospital 1,R,R,NaN,ERP003621,ERS325617,Enterococcus faecium,United Kingdom
3,10678_8#18,ERR369955,Human invasive infection,5,22,Hospital 1,S,S,NaN,ERP003621,ERS325952,Enterococcus faecium,United Kingdom
4,10678_8#19,ERR369956,Human invasive infection,1,132,Hospital 1,R,R,NaN,ERP003621,ERS325629,Enterococcus faecium,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,EFE10021,LN999844.1,Reference strain,4,54,NaN,NaN,not applicable,NaN,NaN,NaN,Enterococcus faecium,United Kingdom
1438,Enterococcus faecium 6E6,CP013994,Reference strain,9,203,NaN,NaN,not applicable,NaN,NaN,NaN,Enterococcus faecium,United Kingdom
1439,NRRL_B_2354,CP004063,Reference strain,5,32,NaN,NaN,not applicable,NaN,NaN,NaN,Enterococcus faecium,United Kingdom
1440,T110,CP006030,Reference strain,10,812,NaN,NaN,not applicable,NaN,NaN,NaN,Enterococcus faecium,United Kingdom


# AAFC Enterococcus Paper Metadata

Raw sequencing data was not deposited for this paper so only a `BioProject` accession (`PRJNA604849`) i.e., `study_accession` above was provided, this study on NCBI contains `sample_accessions` and the raw assembly files.

Therefore, download the metadata from this study and tidy up column names to make merging easier

In [5]:
with open('PRJNA604849_full_biosample_list.xml') as fh:
    PRJNA604849_xml = bs4.BeautifulSoup(fh, 'lxml')

parsed_xml_data = {}
for biosample in PRJNA604849_xml.find_all('biosample'):
    biosample_data = {}
    biosample_data['Sample_name'] = biosample.find(db_label='Sample name').text
    biosample_data['Species'] = biosample.find('organismname').text
    biosample_data['Strain_name'] = biosample.find(attribute_name="strain").text
    biosample_data['Study_accession'] = biosample.find(target="bioproject")['label']
    parsed_xml_data[biosample['accession']] = biosample_data
    
alberta_ncbi = pd.DataFrame(parsed_xml_data).T.reset_index().rename(columns={'index': 'Sample_accession'})

Now we need to parse and tidy up the metadata from https://www.nature.com/articles/s41598-020-61002-5  `41598_2020_61002_MOESM2_ESM.csv`

In [6]:
alberta_metadata = pd.read_csv('41598_2020_61002_MOESM2_ESM.csv', sep='\t')

# drop the 2 inexplicably duplicated rows 
alberta_metadata = alberta_metadata.drop_duplicates()

# get rid of the identical rows apart from _ vs - 
alberta_metadata = alberta_metadata[alberta_metadata['ISOLATE'] != 'SWEntR-0393']
# identical row apart from incorrect species (when compared to NCBI assembly)
alberta_metadata = alberta_metadata[alberta_metadata['ISOLATE'] != 'SWEntR 0262']


# tidy columns up to help with merging later
alberta_metadata = alberta_metadata.rename(columns={'ISOLATION SOURCE': 'Origin',
                                                    'SPECIFIC LOCATION': 'Location',
                                                    'VNCO': 'Vancomycin',
                                                    'AMPI': 'Ampicillin',
                                                    'TEIC': 'Teicoplanin',
                                                    'DOXY': 'Doxycycline',
                                                    'ERTH': 'Erythromycin',
                                                    'GENT': 'Gentamicin',
                                                    'LNZD': 'Linezolid',
                                                    'LVFL': 'Levofloxacin',
                                                    'QUIN': 'Quinolone',
                                                    'STEP': 'Streptomycin',
                                                    'NTRO': 'Nitrofurantoin',
                                                    'TGC': 'Tigecycline',
                                                    'SPECIATION': 'Species',
                                                    'SOURCE CODE': 'Source_code',
                                                    'ISOLATE': 'Isolate_name_paper'})

# get rid of useless columns and add extra column to help with merging UK and AB metadata
alberta_metadata = alberta_metadata.drop(['Unnamed: 18', "Resistance count", 'LOCATION'], axis=1)
alberta_metadata['Province/country'] = 'Alberta/Canada'

# Remove trailing spaces that were left in the paper metadata
alberta_metadata['Species'] = alberta_metadata['Species'].str.strip()

# To ensure we merge the correct identifiers we are going to use the Source_code
# information AS WELL as the isolate_name, therefore let's create a new identifier
# out of the source code and isolate_name (and then remove spaces/underscores/hyphens etc)
def combine_source_and_isolate_name(row):
    """
    Try to combine isolate name with the source code field
    if it isn't already prefixed by the source code information
    """
    # Spaces, hyphens and dashes are a major source of disconnect so just remove them and try to map
    metadata_isolate_name = row['Isolate_name_paper'].replace('_', '').replace('-', '').replace(' ', '')
    if metadata_isolate_name.startswith(row['Source_code']):
        return metadata_isolate_name
    else:
        return row['Source_code'] +  metadata_isolate_name


alberta_metadata['Source_code_and_isolate_name'] = alberta_metadata.apply(\
                                                            combine_source_and_isolate_name, axis=1)

Now we want to merge in the accession data from the bioproject.

Unfortunately the `isolate_name`'s in the NCBI bioproject do not match the `isolate_name`'s in the paper metadata so this needs fixing. 

Fortunately, most are pretty similar and its obvious what the correct `isolate_name` should be for each entry in the paper metadata (the deposited names are the "correct" ones).

Therefore, let's find the closest match between the two sets of names using levenshtein distances,
after manual review this resulted in false positives so let's match all digits in the sample names as a safety check.

After this we can manually review any remaining mismatches

In [7]:
def attempt_to_reconcile_names(biosample_associated_identifiers, paper_metadata_identifiers):
    """
    Take a list of "correct" identifiers from the BioSamples and try to match them to the 
    paper metadata identifiers 
    
    First find the closest matching string using levenshtein distances
    Then as a safety move extract all digits and confirm they match between
    the biosample identifier and the closest paper identifier
    
    returns: dictionary mapping likely paper identifiers to biosample identifiers
             list of biosample identifiers without an obvious match for manual fixing
    """
    closest_match_string_dists = {}
    for deposited_isolate_name in biosample_associated_identifiers:
        # hyphens dashes and spaces are a major source of error so remove
        stripped_deposited_isolate_name = deposited_isolate_name.replace('_', '').replace('-', '').replace(' ', '')
        
        closest_isolate_name = []
        for paper_metadata_isolate_name in paper_metadata_identifiers:
            closest_isolate_name.append((paper_metadata_isolate_name, stringdist.rdlevenshtein_norm(stripped_deposited_isolate_name.lower(),
                                                                   paper_metadata_isolate_name.lower())))
        closest_isolate_name = sorted(closest_isolate_name, key=lambda x: x[1])
        closest_match_string_dists[closest_isolate_name[0][0]] = deposited_isolate_name
        
    # now check the numerical parts match (sans any leading 0s) to avoid false positives
    non_numerically_matching = []
    closest_match_string_dist_and_numerical_match = {}
    for closest_paper_isolate_name, deposited_isolate_name in closest_match_string_dists.items():
        digits_in_paper_name = "".join(re.findall(r'\d+', closest_paper_isolate_name))
        digits_in_deposited_name = "".join(re.findall(r'\d+', deposited_isolate_name))
        if digits_in_deposited_name.lstrip('0') == digits_in_paper_name.lstrip('0'):
            closest_match_string_dist_and_numerical_match[closest_paper_isolate_name] = deposited_isolate_name
        else:
            non_numerically_matching.append(deposited_isolate_name)
            
    return closest_match_string_dist_and_numerical_match, non_numerically_matching


isolate_to_strain_matches, isolate_to_strain_mismatches = attempt_to_reconcile_names(alberta_ncbi['Strain_name'].values, alberta_metadata['Source_code_and_isolate_name'])

Now for manual and semi-manual fixes of missing isolates 

In [8]:
manual_mappings_from_isolate = {
    'WW0145I': 'ES-C-ST001-01FEB16-0145I',
    'FC0145J': 'ES-C-ST001-01FEB16-0145J',
    'WW0145K': 'ES-C-ST001-01FEB16-0145K',
    'WW0154I': 'ES-C-ST001-04JUL16-0154I',
    'WW0134A': 'ES-C-ST001-05OCT15-0134A',
    'WW0134B': 'ES-C-ST001-05OCT15-0134B',
    'WW0134C': 'ES-C-ST001-05OCT15-0134C',
    'WW0134E': 'ES-C-ST001-05OCT15-0134E',
    'WW0134L': 'ES-C-ST001-05OCT15-0134L',
    'WW0141E': 'ES-C-ST001-07DEC15-0141E',
    'WW0141M': 'ES-C-ST001-07DEC15-0141M',
    'FC0096K': 'ES-C-ST001-15JUN15-0096K',
    'WW0039D': 'ES-C-ST001-25AUG14-0039D',
    'WW0039E': 'ES-C-ST001-25AUG14-0039E',
    'FC0117B': 'ES-C-ST001-25AUG15-0117B',
    'WW0117D': 'ES-C-ST001-25AUG15-0117D',
    'WW0117E': 'ES-C-ST001-25AUG15-0117E',
    'WW0117I': 'ES-C-ST001-25AUG15-0117I',
    'WW0117M': 'ES-C-ST001-25AUG15-0117M',
    'WW0081E': 'ES-C-ST001-27APR15-0081E',
    'WW0081L': 'ES-C-ST001-27APR15-0081L',
    'WW0146B': 'ES-C-ST002-01FEB16-0146B',
    'FC0146C': 'ES-C-ST002-01FEB16-0146C',
    'WW0146D': 'ES-C-ST002-01FEB16-0146D',
    'WW0135A': 'ES-C-ST002-05OCT15-0135A',
    'FC0135B': 'ES-C-ST002-05OCT15-0135B',
    'WW0135C': 'ES-C-ST002-05OCT15-0135C',
    'WW0142A': 'ES-C-ST002-07DEC15-0142A',
    'WW0142B': 'ES-C-ST002-07DEC15-0142B',
    'WW0142C': 'ES-C-ST002-07DEC15-0142C',
    'WW0142D': 'ES-C-ST002-07DEC15-0142D',
    'WW0142E': 'ES-C-ST002-07DEC15-0142E',
    'WW0142I': 'ES-C-ST002-07DEC15-0142I',
    'WW0067A': 'ES-M-ST001-03MAR15-0067A',
    'WW0152A': 'ES-M-ST001-03MAY16-0152A',
    'WW0152E': 'ES-M-ST001-03MAY16-0152E',
    'WW0152J': 'ES-M-ST001-03MAY16-0152J',
    'WW0152L': 'ES-M-ST001-03MAY16-0152L',
    'WW0152M': 'ES-M-ST001-03MAY16-0152M',
    'WW0137A': 'ES-M-ST001-03NOV15-0137A',
    'WW0137C': 'ES-M-ST001-03NOV15-0137C',
    'WW0137E': 'ES-M-ST001-03NOV15-0137E',
    'WW0147B': 'ES-M-ST001-08MAR16-0147B',
    'WW0147E': 'ES-M-ST001-08MAR16-0147E',
    'FC0147I': 'ES-M-ST001-08MAR16-0147I',
    'WW0147L': 'ES-M-ST001-08MAR16-0147L',
    'WW0147M': 'ES-M-ST001-08MAR16-0147M',
    'WW0050M': 'ES-M-ST001-13OCT14-0050M',
    'FC0124I': 'ES-M-ST001-15SEP15-0124I',
    'WW0124J': 'ES-M-ST001-15SEP15-0124J',
    'FC0124K': 'ES-M-ST001-15SEP15-0124K',
    'WW0124M': 'ES-M-ST001-15SEP15-0124M',
    'WW0060D': 'ES-M-ST001-23NOV14-0060D',
    'WW0138B': 'ES-M-ST002-03NOV15-0138B',
    'WW0138C': 'ES-M-ST002-03NOV15-0138C',
    'WW0138D': 'ES-M-ST002-03NOV15-0138D',
    'SS0026': 'HC_SS0026',
    'CB0139J': 'CB_0139J',
    'CBEntR0183': 'CB_0183',
    'CBEntR0150': 'CB_0150',
    'CBSWEntR0048': 'CB_0048',
    'CB0127I': 'CB_0127I',
    'CBEntR0158': 'CB_0158',
    'CBEnt0179': 'CB_0179',
    'CBSWEntR0225': 'CB_0225',
    'CBSWEntR0235': 'CB_0235',
    'CBSWEntR0244': 'CB_0244',
    'CBEnt0261': 'CB_0261',
    'CBSWEntR0271': 'CB_0271',
    'CBSWEntR0278': 'CB_0278',
    'CBSWEntR0303': 'CB_0303',
    'CBSWEntR0309': 'CB_0309',
    'CBSWEntR0338': 'CB_0338',
    'CBSWEntR0376': 'CB_0376',
    'CBSWEntR0379': 'CB_0379',
    'CBSWEntR0383': 'CB_0383',
    'CBSWEntR0385': 'CB_0385',
    'CBSWEntR0386': 'CB_0386',
    'CBSWEntR0396': 'CB_0396',
    'CBSWEnt0932': 'CB_0932',
    'CBSWEnt0933': 'CB_0933',
    'CBSWEnt1134': 'CB_1134',
    'CBSWEnt1208': 'CB_1208',
    'CBSWEnt1210': 'CB_1210',
    'CBSWEnt1216': 'CB_1216',
    'NWSEntR0002': 'SW_0002',
    'NWSEnt0025': 'SW_0025',
    'NWSSWEnt1171': 'SWEnt-1171',
    'NWSEnt0202': 'SW_0202',
    'NWSEnt0269': 'SW_0269',
    'NWSEnt0270': 'SW_0270',
    'NWSSWEnR0285': 'SW_0285',
    'NWSSWEntR0295': 'SW_0295',
    'NWSSWEntR0325': 'SW_0325',
    'NWSSWEntR0367': 'SW_0367',
    'NWSSWEntR0374': 'SW_0374',
    'NWSSWEnt0985': 'SW_0985',
    'NWSSWEnt1073': 'SW_1073',
    'CBSWEnt1223': 'CB_1223'}

# update the mapping with the manual fixed
isolate_to_strain_matches.update(manual_mappings_from_isolate)

Unfortunately, this still leaves us with 12 isolates that are in the deposited NCBI data but don't seem to have any supplied metadata

In [9]:
unresolved_ncbi = alberta_ncbi[~alberta_ncbi['Strain_name'].isin(isolate_to_strain_matches.values())].sort_values('Strain_name')['Strain_name'].values
# add a status to the NCBI data indicating missing
alberta_ncbi.loc[alberta_ncbi['Strain_name'].isin(unresolved_ncbi), 'Status'] = 'No metadata in original paper (10.1038/s41598-020-61002-5)'

Time to add the correct mappings to the paper metadata sheet

In [10]:
# translate the incorrect isolate_names in the paper metadata to the correct ones deposited in NCBI
alberta_metadata['Strain_name'] = alberta_metadata['Source_code_and_isolate_name'].apply(lambda x: isolate_to_strain_matches[x] if x in isolate_to_strain_matches else f"UNMATCHED: {x}")

# add the 12 unresolved identifiers to the paper metadata before merging
alberta_metadata = pd.concat([alberta_metadata, pd.DataFrame({'Strain_name': unresolved_ncbi})])

alberta_merged = pd.merge(alberta_ncbi, alberta_metadata, 
                          validate='one_to_one', how='inner', 
                          on='Strain_name', suffixes=['_ncbi', '_paper'])

## Add the read information and identify which reads need to be change

First lets parse the read data from deivos

In [11]:
read_data = pd.read_csv('Enterococcus_reads.tsv', sep='\t')
read_data = read_data.rename(columns={'Sample_name': "Strain_name"})
read_data = read_data.replace({'E_faecalis': 'Enterococcus faecalis',
                               'E_faecium': 'Enterococcus faecium'})

# split the datasets for now
uk_read_data = read_data[read_data['Strain_name'].str.startswith('ERR')]
uk_read_data['Run_accession'] = uk_read_data['Strain_name']
alberta_read_data = read_data[~read_data['Strain_name'].str.startswith('ERR')]


<ipython-input-11-59360e37e34a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uk_read_data['Run_accession'] = uk_read_data['Strain_name']


In [12]:
uk_read_data

,Strain_name,Species,Read_1,Read_2,Run_accession
382,ERR1007500,Enterococcus faecium,E_faecium_342/all_reads/ERR1007500_1.fastq.gz,E_faecium_342/all_reads/ERR1007500_2.fastq.gz,ERR1007500
383,ERR1007501,Enterococcus faecium,E_faecium_342/all_reads/ERR1007501_1.fastq.gz,E_faecium_342/all_reads/ERR1007501_2.fastq.gz,ERR1007501
384,ERR1036024,Enterococcus faecium,E_faecium_342/all_reads/ERR1036024_1.fastq.gz,E_faecium_342/all_reads/ERR1036024_2.fastq.gz,ERR1036024
385,ERR1036025,Enterococcus faecium,E_faecium_342/all_reads/ERR1036025_1.fastq.gz,E_faecium_342/all_reads/ERR1036025_2.fastq.gz,ERR1036025
386,ERR1036026,Enterococcus faecium,E_faecium_342/all_reads/ERR1036026_1.fastq.gz,E_faecium_342/all_reads/ERR1036026_2.fastq.gz,ERR1036026
...,...,...,...,...,...
1045,ERR987645,Enterococcus faecium,E_faecium_342/all_reads/ERR987645_1.fastq.gz,E_faecium_342/all_reads/ERR987645_2.fastq.gz,ERR987645
1046,ERR987646,Enterococcus faecium,E_faecium_342/all_reads/ERR987646_1.fastq.gz,E_faecium_342/all_reads/ERR987646_2.fastq.gz,ERR987646
1047,ERR987647,Enterococcus faecium,E_faecium_342/all_reads/ERR987647_1.fastq.gz,E_faecium_342/all_reads/ERR987647_2.fastq.gz,ERR987647
1048,ERR987650,Enterococcus faecium,E_faecium_342/all_reads/ERR987650_1.fastq.gz,E_faecium_342/all_reads/ERR987650_2.fastq.gz,ERR987650


In [13]:
uk_metadata.loc[(uk_metadata['Origin'] != 'Reference strain') & (~uk_metadata['Status'].isna())]

KeyError: 'Status'

In [ ]:
uk_metadata.loc[~uk_metadata['Status'].isna()]

In [ ]:
df = pd.merge(uk_metadata, uk_read_data, on='Run_accession', how='left')
df[df['Strain_name'].isna()]['Origin'].value_counts()

Now merge the read information with the UK metadata as this should be easy

In [ ]:
pd.merge(uk_metadata, read_data, validate="one_to_one", how='left')

In [ ]:
read_dataa

In [ ]:
alberta_read_data

Identify what read information isn't readily mappable to the merged alberta metadata

In [ ]:
#alberta_metadata = alberta_metadata.drop(['Isolate_name_paper', 'Species_paper', 'Source_code_and_isolate_name'], axis=1)

In [ ]:
alberta_read_data[alberta_read_data['Strain_name'].isin(alberta_merged['Isolate_name_paper'])]



In [ ]:
alberta_read_data

# Merging the two datasets

In [ ]:
alberta_metadata